In [1]:
import os
import time
from datetime import date, datetime
import matplotlib.patches as mpatches
from math import log
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb 
import xgboost as xgb 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [2]:
stock = pd.read_csv('C:\\Users\\frede\\OneDrive\\桌面\\RL\\stock1.csv')

In [3]:
stock['close_diff'] = None
for i in range(1,len(stock['close'])):
    stock['close_diff'][i] = stock['close'][i] - stock['close'][i-1]


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
stock['year'] = None
stock['month'] = None
stock['day'] = None
for i in range(len(stock['Date'])):
    stock['year'][i] = stock['Date'][i].split('-')[0]
    stock['month'][i] = stock['Date'][i].split('-')[1]
    stock['day'][i] = stock['Date'][i].split('-')[2]


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
stock['Date'] = stock['Date'].str.replace('-','')

In [6]:
stock

,Date,open,high,low,close,volume,MA5,MA10,MA20,y,...,MACD,MACDsignal,MACDhist,RSI5,RSI10,RSI20,close_diff,year,month,day
0,19801212,0.41,0.41,0.41,0.41,117258400.0,NaN,NaN,NaN,0.40,...,NaN,NaN,NaN,NaN,NaN,NaN,None,1980,12,12
1,19801215,0.39,0.39,0.39,0.39,43971200.0,NaN,NaN,NaN,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,1980,12,15
2,19801216,0.36,0.36,0.36,0.36,26432000.0,NaN,NaN,NaN,0.44,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,1980,12,16
3,19801217,0.37,0.37,0.37,0.37,21610400.0,NaN,NaN,NaN,0.46,...,NaN,NaN,NaN,NaN,NaN,NaN,0.01,1980,12,17
4,19801218,0.38,0.38,0.38,0.38,18362400.0,0.382,NaN,NaN,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,0.01,1980,12,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,20200403,242.80,245.70,238.97,241.41,32470000.0,247.270,245.930,251.4100,NaN,...,-3.068963,-4.433047,1.364085,41.042554,42.571047,42.721484,-3.52,2020,04,03
9911,20200406,250.90,263.11,249.38,262.47,50455100.0,248.802,249.740,251.2250,NaN,...,-0.364401,-3.619318,3.254917,65.266910,53.811856,48.607410,21.06,2020,04,06
9912,20200407,270.80,271.70,259.00,259.43,50721800.0,249.830,250.995,249.9295,NaN,...,0.885362,-2.718382,3.603744,60.762147,52.173935,47.860121,-3.04,2020,04,07
9913,20200408,262.74,267.37,261.23,266.07,42223800.0,254.862,253.050,249.4615,NaN,...,2.477401,-1.679225,4.156626,66.983857,55.463814,49.640207,6.64,2020,04,08


In [7]:
stock.shift(-5)

,Date,open,high,low,close,volume,MA5,MA10,MA20,y,...,MACD,MACDsignal,MACDhist,RSI5,RSI10,RSI20,close_diff,year,month,day
0,19801219,0.40,0.40,0.40,0.40,12157600.0,0.380,NaN,NaN,0.51,...,NaN,NaN,NaN,44.444444,NaN,NaN,0.02,1980,12,19
1,19801222,0.42,0.42,0.42,0.42,9340800.0,0.386,NaN,NaN,0.50,...,NaN,NaN,NaN,56.521739,NaN,NaN,0.02,1980,12,22
2,19801223,0.44,0.44,0.44,0.44,11737600.0,0.402,NaN,NaN,0.48,...,NaN,NaN,NaN,65.811966,NaN,NaN,0.02,1980,12,23
3,19801224,0.46,0.46,0.46,0.46,12000800.0,0.420,NaN,NaN,0.49,...,NaN,NaN,NaN,73.018550,NaN,NaN,0.02,1980,12,24
4,19801226,0.50,0.50,0.50,0.50,13893600.0,0.444,0.413,NaN,0.48,...,NaN,NaN,NaN,82.330204,NaN,NaN,0.04,1980,12,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# stock['diff_rate'] = (stock['close'].shift(1)/stock['close'])-1
stock['diff_rate'] = None
for i in range(1,len(stock['close'])):
    stock['diff_rate'][i] = (stock['close'][i]/stock['close'][i-1]) - 1


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# 五日後漲標記 1，反之標記 0
stock['y'] = np.where(stock.close.shift(-5) > stock.close, 1, 0)

In [10]:
stock_data = stock[['volume', 'MA5', 'MA10', 'MA20', 'k', 'd', 'MACD', 'MACDsignal', 'MACDhist', 'RSI5', 'RSI10',
       'RSI20', 'close_diff', 'year', 'month', 'day', 'diff_rate']]
stock_data['close_diff'] = pd.to_numeric(stock_data['close_diff'])
stock_data['year'] = pd.to_numeric(stock_data['year'])
stock_data['month'] = pd.to_numeric(stock_data['month'])
stock_data['day'] = pd.to_numeric(stock_data['day'])
stock_y = stock['y']


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [11]:
train_date_min = 20060101
train_date_max = 20151231
val_date_min = 20160101
val_date_max = 201801231
test_date_min = 20160101
test_date_max = int(stock['Date'][9914])

In [12]:
def idx_range(df,minn,maxx):
    index = []
    for i in range(len(df)):
        if minn <= int(df[i]) & int(df[i]) <= maxx:
            index.append(i)
    
    return index[0], index[-1]

In [13]:
stock['Date'][8839]

'20151231'

In [14]:
train_data_idx = idx_range(stock['Date'],train_date_min,train_date_max)
val_data_idx = idx_range(stock['Date'],val_date_min,val_date_max)
test_data_idx = idx_range(stock['Date'],test_date_min,test_date_max)

In [15]:
train_data = stock_data[['volume', 'MA5', 'MA10', 'MA20', 'k', 'd', 'MACD', 'MACDsignal', 'MACDhist', 'RSI5', 'RSI10',
       'RSI20', 'close_diff', 'year', 'month', 'day', 'diff_rate']][train_data_idx[0]:train_data_idx[1]+1].values
train_y = stock_y[train_data_idx[0]:train_data_idx[1]+1].values
val_data = stock_data[['volume', 'MA5', 'MA10', 'MA20', 'k', 'd', 'MACD', 'MACDsignal', 'MACDhist', 'RSI5', 'RSI10',
       'RSI20', 'close_diff', 'year', 'month', 'day', 'diff_rate']][val_data_idx[0]:val_data_idx[1]+1].values
val_y = stock_y[val_data_idx[0]:val_data_idx[1]+1].values
test_data = stock_data[['volume', 'MA5', 'MA10', 'MA20', 'k', 'd', 'MACD', 'MACDsignal', 'MACDhist', 'RSI5', 'RSI10',
       'RSI20', 'close_diff', 'year', 'month', 'day', 'diff_rate']][test_data_idx[0]:test_data_idx[1]+1-5].values
test_y = stock_y[test_data_idx[0]:test_data_idx[1]+1-5].values

In [16]:
print('train上漲：',train_y.sum())
print('train下跌：',len(train_y) - train_y.sum())
print('val上漲：',val_y.sum())
print('val上漲：',val_y.sum())
print('test上漲：',test_y.sum())
print('test下跌：',len(test_y) - test_y.sum())

train上漲： 1409
train下跌： 1108
val上漲： 655
val上漲： 655
test上漲： 655
test下跌： 415


In [17]:
""" First import all the required libraries """
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import fbeta_score, make_scorer

In [18]:
""" LightGBM Implementation and Results"""
# We need to convert our training data into LightGBM dataset format
d_train = lgb.Dataset(train_data, label=train_y)
# Setting parameters for training
# objective set to binary for binary classification problem
# boosting_type set to gbdt for gradient boosting
# binary_logloss as metric for binary classification predictions
# other parameters randomly selected and subject to change for optimization
params = {'boosting_type': 'gbdt',
          'colsample_bytree': 0.58, 'n_estimators': 75, 'subsample': 0.4,
          'learning_rate': 0.0001,
          'max_depth': 15,
          'metric': 'binary_logloss',
          'min_data': 50,
          'num_leaves': 10,
          'objective': 'binary',
          'sub_feature': 0.5}
# fit the clf_LGBM on training data with 100 training iterations
clf_LGBM = lgb.train(params, d_train, 100)
# make predictions with test data
y_pred = clf_LGBM.predict(val_data)
# sinec the output is a list of probabilities, below we have converted the probabilities
# to binary prediction with threshold set at 0.5
for i in range(0, len(y_pred)):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else:  
        y_pred[i]=0
# evaluate predictions with accuracy metric
clf_LGBM_accuracy = accuracy_score(val_y, y_pred)
# evaluate predictions with F1-score metric
clf_LGBM_f1 = f1_score(val_y, y_pred)
print("LightGBM Classifier [Accuracy score: {:.4f}, f1-score: {:.4f}]".format(clf_LGBM_accuracy, clf_LGBM_f1))

LightGBM Classifier [Accuracy score: 0.6093, f1-score: 0.7572]


Found `n_estimators` in params. Will use it instead of argument


In [19]:
""" Optimization of LightGBM """
# Choose LGBM Classifier as the algorithm for optimization with GridSearch
clf_LGBM2 = lgb.LGBMClassifier(boosting_type = 'gbdt', metric = 'binary_logloss', 
                               min_data = 50, objective = 'binary', sub_feature = 0.5)
# Create a dictionary for the parameters
gridParams = {'learning_rate': [0.0001, 0.0003, 0.0005, 0.001],'n_estimators': [75, 100, 125],
             'num_leaves': [15, 16, 17],'colsample_bytree' : [0.58, 0.60, 0.62],'subsample' : [0.4, 0.5, 0.7]}
# Choose the time series cross-validator
tscv = TimeSeriesSplit(n_splits=3)
# Create the GridSearch object
grid = GridSearchCV(clf_LGBM2, gridParams, verbose=1, cv= tscv)
# Fit the grid search object to the data to compute the optimal model
grid_fit_LGBM = grid.fit(train_data, train_y)
# Return the optimal model after fitting the data
best_clf_LGBM = grid_fit_LGBM.best_estimator_
# Make predictions with the optimal model
best_predictions_LGBM = best_clf_LGBM.predict(val_data)
# Get the accuracy and F1_score of the optimized model
clf_LGBM_optimized_accuracy = accuracy_score(val_y, best_predictions_LGBM)
clf_LGBM_optimized_f1 = f1_score(val_y, best_predictions_LGBM)
print("LGBM Classifier Optimized [Accuracy score: {:.4f}, f1-score: {:.4f}]".format(clf_LGBM_optimized_accuracy, clf_LGBM_optimized_f1))
print(grid_fit_LGBM.best_params_)
print(grid_fit_LGBM.best_score_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


LGBM Classifier Optimized [Accuracy score: 0.6093, f1-score: 0.7572]
{'colsample_bytree': 0.58, 'learning_rate': 0.0001, 'n_estimators': 75, 'num_leaves': 15, 'subsample': 0.4}
0.5548489666136724


[Parallel(n_jobs=1)]: Done 972 out of 972 | elapsed:   27.4s finished


In [20]:
best_predictions_LGBM = best_clf_LGBM.predict(test_data)
# Get the accuracy and F1_score of the optimized model
clf_LGBM_optimized_accuracy = accuracy_score(test_y, best_predictions_LGBM)
clf_LGBM_optimized_f1 = f1_score(test_y, best_predictions_LGBM)
print("LGBM Classifier Optimized [Accuracy score: {:.4f}, f1-score: {:.4f}]".format(clf_LGBM_optimized_accuracy, clf_LGBM_optimized_f1))
print(grid_fit_LGBM.best_params_)
print(grid_fit_LGBM.best_score_)

LGBM Classifier Optimized [Accuracy score: 0.6121, f1-score: 0.7594]
{'colsample_bytree': 0.58, 'learning_rate': 0.0001, 'n_estimators': 75, 'num_leaves': 15, 'subsample': 0.4}
0.5548489666136724


In [21]:
print(pd.DataFrame({
        'column': ['volume', 'MA5', 'MA10', 'MA20', 'k', 'd', 'MACD', 'MACDsignal', 'MACDhist', 'RSI5', 'RSI10',
       'RSI20', 'close_diff', 'year', 'month', 'day', 'diff_rate'],
        'importance': clf_LGBM.feature_importance(),
    }).sort_values(by='importance'))

        column  importance
12  close_diff           0
16   diff_rate           1
5            d           6
4            k          11
0       volume          18
15         day          21
13        year          21
11       RSI20          28
7   MACDsignal          41
10       RSI10          43
6         MACD          52
2         MA10          53
9         RSI5          58
8     MACDhist          59
1          MA5          79
3         MA20          79
14       month         105


In [22]:
stock_data = stock[['volume', 'MA5', 'MA10', 'MA20', 'k', 'd', 'MACD', 'MACDsignal', 'MACDhist', 'RSI5', 'RSI10',
       'RSI20', 'close_diff', 'year', 'month', 'day', 'diff_rate']]
stock_data['close_diff'] = pd.to_numeric(stock_data['close_diff'])
stock_data['year'] = pd.to_numeric(stock_data['year'])
stock_data['month'] = pd.to_numeric(stock_data['month'])
stock_data['day'] = pd.to_numeric(stock_data['day'])
stock_y = stock['y'].shift(-30)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [23]:
def load_data(price_data):
    data = []
    for i in range(len(price_data)-5):
        if price_data[i+5] > price_data[i] :
            data.append(1)
        else:
            data.append(0)
    return data
up_down = np.array(load_data(stock.close))
up_down = pd.DataFrame(up_down)

In [24]:
#定義正規化函式
def normalize(train):
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

In [25]:
norm_data = normalize(stock_data)

In [26]:
norm_data

,volume,MA5,MA10,MA20,k,d,MACD,MACDsignal,MACDhist,RSI5,RSI10,RSI20,close_diff,year,month,day,diff_rate
0,0.016956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.502292,0.498748,-0.124690,NaN
1,-0.022550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000717,-0.502292,0.498748,-0.024690,-0.0578321
2,-0.032005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000870,-0.502292,0.498748,0.008643,-0.0904674
3,-0.034604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000259,-0.502292,0.498748,0.041977,0.0309482
4,-0.036355,-0.093556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000259,-0.502292,0.498748,0.075310,0.0300776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,-0.028750,0.668990,0.667766,0.691831,-0.335503,-0.194029,-0.155892,-0.280944,0.132091,-0.123109,-0.125317,-0.145600,-0.054153,0.497708,-0.228524,-0.424690,-0.0179299
9911,-0.019055,0.673722,0.679583,0.691252,-0.035709,-0.194145,-0.021859,-0.230254,0.315155,0.125615,0.008442,-0.061946,0.321115,0.497708,-0.228524,-0.324690,0.0999002
9912,-0.018911,0.676897,0.683475,0.687194,0.075498,-0.098585,0.040077,-0.174131,0.348927,0.079362,-0.011049,-0.072567,-0.046824,0.497708,-0.228524,-0.291357,-0.0146954
9913,-0.023492,0.692439,0.689848,0.685728,0.296157,0.111968,0.118976,-0.109398,0.402455,0.143244,0.028099,-0.047268,0.100962,0.497708,-0.228524,-0.258023,0.0284164


In [27]:
def train_windows(df, df1, ref_day=5, predict_day=1):
    X_train = []
    Y_train = []
    for i in range(df.shape[0]-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:]))
        Y_train.append(df1.iloc[i])
    return np.array(X_train), np.array(Y_train)
X, Y=train_windows(norm_data,up_down,5,1)

In [39]:
X_, Y_=train_windows(stock,up_down,5,1)

In [44]:
X_[9909]

array([['20200402', 240.34, 245.15, 236.9, 244.93, 41483500.0,
        248.5360000000001, 244.71299999999994, 253.79099999999968, 1,
        31.30653132693181, 50.654024677570035, -2.7256757334917268,
        -4.774068371404748, 2.04839263791302, 45.26373015468791,
        44.188775790640705, 43.512779835429775, 4.02000000000001, '2020',
        '04', '02', 0.01668672948403982],
       ['20200403', 242.8, 245.7, 238.97, 241.41, 32470000.0,
        247.27000000000007, 245.92999999999992, 251.40999999999968, 0,
        18.87446000417492, 33.02327288202877, -3.0689625766642616,
        -4.43304721245665, 1.3640846357923877, 41.04255436495647,
        42.57104654197871, 42.72148365373275, -3.5200000000000102,
        '2020', '04', '03', -0.014371453068223627],
       ['20200406', 250.9, 263.11, 249.38, 262.47, 50455100.0,
        248.8020000000001, 249.73999999999995, 251.22499999999968, 0,
        48.85390029936709, 33.01163054349113, -0.3644013246623956,
        -3.6193180348977987, 3.25

In [43]:
stock

,Date,open,high,low,close,volume,MA5,MA10,MA20,y,...,MACDsignal,MACDhist,RSI5,RSI10,RSI20,close_diff,year,month,day,diff_rate
0,19801212,0.41,0.41,0.41,0.41,117258400.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,None,1980,12,12,None
1,19801215,0.39,0.39,0.39,0.39,43971200.0,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,-0.02,1980,12,15,-0.0487805
2,19801216,0.36,0.36,0.36,0.36,26432000.0,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,-0.03,1980,12,16,-0.0769231
3,19801217,0.37,0.37,0.37,0.37,21610400.0,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,0.01,1980,12,17,0.0277778
4,19801218,0.38,0.38,0.38,0.38,18362400.0,0.382,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,0.01,1980,12,18,0.027027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,20200403,242.80,245.70,238.97,241.41,32470000.0,247.270,245.930,251.4100,0,...,-4.433047,1.364085,41.042554,42.571047,42.721484,-3.52,2020,04,03,-0.0143715
9911,20200406,250.90,263.11,249.38,262.47,50455100.0,248.802,249.740,251.2250,0,...,-3.619318,3.254917,65.266910,53.811856,48.607410,21.06,2020,04,06,0.0872375
9912,20200407,270.80,271.70,259.00,259.43,50721800.0,249.830,250.995,249.9295,0,...,-2.718382,3.603744,60.762147,52.173935,47.860121,-3.04,2020,04,07,-0.0115823
9913,20200408,262.74,267.37,261.23,266.07,42223800.0,254.862,253.050,249.4615,0,...,-1.679225,4.156626,66.983857,55.463814,49.640207,6.64,2020,04,08,0.0255946


In [29]:
train_data_idx_new = train_data_idx[0]-5, train_data_idx[1]-5
val_data_idx_new = val_data_idx[0]-5, val_data_idx[1]-5
test_data_idx_new = test_data_idx[0]-5, test_data_idx[1]-5

In [30]:
train_data = X[train_data_idx_new[0]:train_data_idx_new[1]+1]
train_y = Y[train_data_idx_new[0]:train_data_idx_new[1]+1]
val_data = X[val_data_idx_new[0]:val_data_idx_new[1]+1]
val_y = Y[val_data_idx_new[0]:val_data_idx_new[1]+1]
test_data = X[test_data_idx_new[0]:test_data_idx_new[1]+1]
test_y = Y[test_data_idx_new[0]:test_data_idx_new[1]+1]

In [31]:
print(test_data.shape)
print(test_y.shape)

(1075, 5, 17)
(1075, 1)


In [32]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth =True

set_session(tf.Session(config=config)) 

#======================================================================
config.gpu_options.per_process_gpu_memory_fraction = 0.8

Using TensorFlow backend.


In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
regressor = Sequential()
regressor.add(LSTM(units = 256, return_sequences = True, input_shape = (train_data.shape[1], train_data.shape[2])))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 256, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 256, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 256))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1, activation='sigmoid'))
regressor.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
filepath = 'weights-best_.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
regressor.fit(train_data, train_y, verbose=2, callbacks=[checkpoint], validation_data = (val_data,val_y), epochs = 200, batch_size = 32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 2517 samples, validate on 1075 samples
Epoch 1/200
 - 5s - loss: 0.5850 - acc: 0.6830 - val_loss: 0.6211 - val_acc: 0.6893

Epoch 00001: val_loss improved from inf to 0.62106, saving model to weights-best_.hdf5
Epoch 2/200
 - 3s - loss: 0.3811 - acc: 0.8256 - val_loss: 0.4071 - val_acc: 0.8177

Epoch 00002: val_loss improved from 0.62106 to 0.40711, saving model to weights-best_.hdf5
Epoch 3/200
 - 3s - loss: 0.3754 - acc: 0.8343 - val_loss: 0.4755 - val_acc: 0.7916

Epoch 00003: val_loss did not improve from 0.40711
Epoch 4/200
 - 3s - loss: 0.3647 - acc: 0.8415 - val_loss: 0.4368 - val_acc: 0.8121

Epoch 00004: val_loss did not improve from 0.40711
Epoch 5/200
 - 3s - loss: 0.3673 - acc: 0.8304 - val_loss: 0.4069 - val_acc: 0.8270

Epoch 00005:


Epoch 00051: val_loss did not improve from 0.35718
Epoch 52/200
 - 3s - loss: 0.2924 - acc: 0.8784 - val_loss: 0.5396 - val_acc: 0.8112

Epoch 00052: val_loss did not improve from 0.35718
Epoch 53/200
 - 3s - loss: 0.2901 - acc: 0.8780 - val_loss: 0.5098 - val_acc: 0.8019

Epoch 00053: val_loss did not improve from 0.35718
Epoch 54/200
 - 3s - loss: 0.2949 - acc: 0.8717 - val_loss: 0.5181 - val_acc: 0.7944

Epoch 00054: val_loss did not improve from 0.35718
Epoch 55/200
 - 3s - loss: 0.2884 - acc: 0.8689 - val_loss: 0.5168 - val_acc: 0.8056

Epoch 00055: val_loss did not improve from 0.35718
Epoch 56/200
 - 3s - loss: 0.2908 - acc: 0.8752 - val_loss: 0.5233 - val_acc: 0.8251

Epoch 00056: val_loss did not improve from 0.35718
Epoch 57/200
 - 3s - loss: 0.2890 - acc: 0.8780 - val_loss: 0.4611 - val_acc: 0.8158

Epoch 00057: val_loss did not improve from 0.35718
Epoch 58/200
 - 3s - loss: 0.2860 - acc: 0.8784 - val_loss: 0.5130 - val_acc: 0.8298

Epoch 00058: val_loss did not improve fr


Epoch 00111: val_loss did not improve from 0.35718
Epoch 112/200
 - 3s - loss: 0.1239 - acc: 0.9480 - val_loss: 1.0760 - val_acc: 0.7953

Epoch 00112: val_loss did not improve from 0.35718
Epoch 113/200
 - 3s - loss: 0.1072 - acc: 0.9515 - val_loss: 1.1856 - val_acc: 0.7842

Epoch 00113: val_loss did not improve from 0.35718
Epoch 114/200
 - 3s - loss: 0.1010 - acc: 0.9555 - val_loss: 1.1872 - val_acc: 0.7879

Epoch 00114: val_loss did not improve from 0.35718
Epoch 115/200
 - 3s - loss: 0.0932 - acc: 0.9615 - val_loss: 1.1567 - val_acc: 0.7963

Epoch 00115: val_loss did not improve from 0.35718
Epoch 116/200
 - 3s - loss: 0.1200 - acc: 0.9511 - val_loss: 1.2058 - val_acc: 0.7926

Epoch 00116: val_loss did not improve from 0.35718
Epoch 117/200
 - 3s - loss: 0.1299 - acc: 0.9484 - val_loss: 1.2470 - val_acc: 0.7963

Epoch 00117: val_loss did not improve from 0.35718
Epoch 118/200
 - 3s - loss: 0.1217 - acc: 0.9507 - val_loss: 1.3124 - val_acc: 0.7860

Epoch 00118: val_loss did not imp

 - 3s - loss: 0.0674 - acc: 0.9797 - val_loss: 1.5349 - val_acc: 0.7786

Epoch 00171: val_loss did not improve from 0.35718
Epoch 172/200
 - 3s - loss: 0.0710 - acc: 0.9746 - val_loss: 1.2259 - val_acc: 0.8056

Epoch 00172: val_loss did not improve from 0.35718
Epoch 173/200
 - 3s - loss: 0.0501 - acc: 0.9817 - val_loss: 1.2118 - val_acc: 0.8000

Epoch 00173: val_loss did not improve from 0.35718
Epoch 174/200
 - 3s - loss: 0.0209 - acc: 0.9932 - val_loss: 1.4200 - val_acc: 0.7842

Epoch 00174: val_loss did not improve from 0.35718
Epoch 175/200
 - 3s - loss: 0.0155 - acc: 0.9956 - val_loss: 1.4549 - val_acc: 0.7916

Epoch 00175: val_loss did not improve from 0.35718
Epoch 176/200
 - 3s - loss: 0.0422 - acc: 0.9869 - val_loss: 1.4544 - val_acc: 0.7907

Epoch 00176: val_loss did not improve from 0.35718
Epoch 177/200
 - 3s - loss: 0.0497 - acc: 0.9825 - val_loss: 1.4798 - val_acc: 0.7851

Epoch 00177: val_loss did not improve from 0.35718
Epoch 178/200
 - 3s - loss: 0.0294 - acc: 0.9909

In [34]:
from keras.models import load_model
regressor = load_model(filepath)

In [35]:
predict_y = regressor.predict(test_data)

In [36]:
regressor.evaluate(test_data,test_y)

1075/1075 [==============================] - 1s 1ms/step


[0.35718299954436544, 0.8344186048729475]